# ```jzon```

## Contents:
* [Description](#description)
* [Breakdown](#breakdown)
* [Exploration](#exploration)

&#10071;: <b>Note that 'Exploration' functionality is still in development.</b>

## Setup

In [3]:
# standard
import sys

# data science
import pandas as pd

# module
from src import breakdown, merge

## Description
<a class="anchor" id="description"></a>

This module was designed to help you handle JSON files. Though there exists some functionality in common packages to help handle ```.json``` files, there are many cases where existing functionality is not flexible. 

The current implementation helps you explore and break down deeply nested files into individual tables. Here 'nested' not only refers to nested objects (```{"a":{"b":"c"}}```) but also nested arrays (```{"a":{"b":["c"]}```) and complex files with different types in arrays. This is important because existing functionality (e.g. Panda's ```DataFrame.json_normalize()```) and would likely either break or improperly convert data of such forms.

Future implementation will also enable the key mapping and exploration of complex files by providing functions to generate graph diagrams of keys and their respective value types over all documents.  

## Breakdown 
<a class="anchor" id="breakdown"></a>

### Getting Atomic Tables

Say we want to break this down into flat files, with each subarray represented as a new table (to avoid extremely complex files). In this case, Panda's ```json_normalize()``` functionality won't unpack the nested subarrays. Even if we add specific commands to unpack one of them through all levels of the nested hierarchy, it wouldn't do so as desired.

This is easiest to see with mock JSON data that has a slightly confusing schema (noting there are much more complex formats in the wild!):

In [4]:
data = \
[{'animals': [{'name': 'faith', 'type': 'cat'},
              {'name': 'shadow', 'type': 'doge'}],
  'date': '2021-01-01',
  'other': {'mood': 'happy'},
  'people': [{'interests': [{'geetar': {'favorite': 'van halen',
                                        'type': 'frankenstrat'}},
                            'kittens',
                            'sillyness'],
              'name': 'dave'},
             {'interests': ['horses', 'painting', 'mma'], 'name': 'becca'}],
  'user_id': 'FDSA1234'},
 {'animals': [{'name': 'felix', 'type': 'ardvark'}],
  'date': '2021-01-02',
  'people': [{'interests': ['motorcycles',
                            {'geetar': {'favorite': 'hendrix',
                                        'type': 'fender'}}],
              'name': 'mike'},
             {'interests': ['reading', 'writing'], 'name': 'tom'}],
  'user_id': 'ASDF4321'}]

In [5]:
pd.json_normalize(data) # people/animals still compacted

,animals,date,people,user_id,other.mood
0,"[{'name': 'faith', 'type': 'cat'}, {'name': 's...",2021-01-01,[{'interests': [{'geetar': {'favorite': 'van h...,FDSA1234,happy
1,"[{'name': 'felix', 'type': 'ardvark'}]",2021-01-02,"[{'interests': ['motorcycles', {'geetar': {'fa...",ASDF4321,NaN


This is where the current module comes in.

In [6]:
table_tag2df = breakdown.dev(data)

print(list(table_tag2df.keys())) # the atomic tables that were created 
display(table_tag2df["root_0<animals_1"]) # an example

['root_0', 'root_0<animals_1', 'root_0<people_1', 'root_0<people_1<interests_2']


,PK,FK,animals.subarray_IDX,animals.name,animals.type
0,0,0,0,faith,cat
1,1,0,1,shadow,doge
2,2,1,0,felix,ardvark


### Getting Merged Tables for Each Table Chain

What if we want to do some cleaning and get rid of all of the individual atomic tables? To do this, we can merge them together. This is done so that each unique 'chain' from root node 'A' to the deepest child nodes are created. Note that redundant chains are ignored. In other words, a merge of parent table A and child table B will not be created if there is also a merge of parent table A with child table B that has child table C. However, if there is another possible child of table B (call it C2), then two merges would be made: A-B-C and A-B-C2. 

In [7]:
# merging
tag2df = merge.merge_tables(table_tag2df)


 ✅ creating table over chain 'root_0<people_1<interests_2'

 ✅ creating table over chain 'root_0<animals_1'
❌ skipping root_0<people_1 since it has subtables


In [8]:
# the resulting tables
for merged_tag, merged_tag_df in tag2df.items():
    print(f"Merged table tag: {merged_tag}")
    display(merged_tag_df.head(3))

Merged table tag: root_0<people_1<interests_2


,root.date,root.user_id,root.other_mood,people.subarray_IDX,people.name,interests.subarray_IDX,interests.interests_geetar_favorite,interests.interests_geetar_type,interests.interests
0,2021-01-01,FDSA1234,happy,0,dave,0,,,kittens
1,2021-01-01,FDSA1234,happy,0,dave,1,,,sillyness
2,2021-01-01,FDSA1234,happy,0,dave,2,van halen,frankenstrat,


Merged table tag: root_0<animals_1


,root.date,root.user_id,root.other_mood,animals.subarray_IDX,animals.name,animals.type
0,2021-01-01,FDSA1234,happy,0,faith,cat
1,2021-01-01,FDSA1234,happy,1,shadow,doge
2,2021-01-02,ASDF4321,,0,felix,ardvark


In [9]:
# compare with the original data
pd.DataFrame(data)

,animals,date,other,people,user_id
0,"[{'name': 'faith', 'type': 'cat'}, {'name': 's...",2021-01-01,{'mood': 'happy'},[{'interests': [{'geetar': {'favorite': 'van h...,FDSA1234
1,"[{'name': 'felix', 'type': 'ardvark'}]",2021-01-02,NaN,"[{'interests': ['motorcycles', {'geetar': {'fa...",ASDF4321


Each merged table is associated with a compound tag of a specific format:
- the ```<``` delimiter separates a parent (left) from a child (right) table where the right table is a subarray under the parent table (```{a:[{b:c}]}``` $\rightarrow$ ```a<b```)
    - to track what index of the array the row is respective to, there is a corresponding ```subarray_IDX``` feature
    - the ```FK``` of the child table on the right of a ```<``` maps to the PK of the table on the left of a ```<``` (the parent table)
- the items inbetween the ```<``` delimiters give lower-level table details
    - the left of a ```_``` is the 'overall feature' the table describes, being the key in the original object that maps to the subarray of information the table represents 
    - the right is the subarray 'level'
- a ```.``` in a column name indicates that the feature was found in a nested object (```{a:{b:c}}``` $\rightarrow$ ```a.b=c```)